In [4]:
import pandas as pd
import numpy as np
import os
import json

# Make sure the output directory exists
# This directory will store the generated HTML file
os.makedirs('../assets', exist_ok=True)

# Load data from CSV files
# Assuming these files are located in a 'data' directory one level up from the script
try:
    socioeconomic = pd.read_csv('../data/socioeconomic.csv')
    consumption = pd.read_csv('../data/Consumption.csv')
    regions = pd.read_csv('../data/Region.csv')
except FileNotFoundError as e:
    print(f"Error loading data file: {e}")
    print("Please ensure 'socioeconomic.csv', 'Consumption.csv', and 'Region.csv' are in the '../data/' directory.")
    # In a real script, you might handle this more gracefully, but for this example, we'll exit
    exit()

# Define translations for Danish to English socioeconomic statuses
# This is used for clearer labeling in the radar charts
socio_translations = {
    "Gennemsnitshusstand": "Average Household",
    "Selvstændig": "Self-employed",
    "Lønmodtager på højeste niveau": "High Income",
    "Lønmodtager på mellemniveau": "Medium Income",
    "Lønmodtager på grundniveau": "Basic Income",
    "Arbejdsløs": "Unemployed",
    "Uddannelsessøgende": "Student",
    "Pensionist, efterlønsmodtager": "Pensioner",
    "Ude af erhverv i øvrigt": "Not in Workforce"
}

# Clean and process socioeconomic data
# Renaming columns and adding a translated group column
socio_df = socioeconomic.copy()
socio_df.columns = [col.strip() for col in socio_df.columns] # Remove leading/trailing whitespace from column names
socio_df = socio_df.rename(columns={
    'Socioøkonomisk status': 'Group',
    '09.8 Pakkerejser': 'Packages',
    '11.1 Restaurationstjenester': 'Restaurants',
    '11.2 Overnatningsfaciliteter': 'Accommodation'
})

# Add translated names
socio_df['Group_EN'] = socio_df['Group'].map(socio_translations)

# Handle any remaining NaN values
socio_df = socio_df.dropna(subset=['Group_EN'])  # Remove rows where Group_EN is NaN

# Calculate total spending
socio_df['Total'] = socio_df['Packages'] + socio_df['Restaurants'] + socio_df['Accommodation']

# Process consumption/age data
age_df = consumption.copy()
# Find the exact column names containing 'Package Holidays - Fixed Prices', etc.
try:
    package_col_age = [col for col in age_df.columns if 'Package Holidays - Fixed Prices' in col][0]
    restaurant_col_age = [col for col in age_df.columns if 'Restaurant Services - Fixed Prices' in col][0]
    accommodation_col_age = [col for col in age_df.columns if 'Accommodation Services - Fixed Prices' in col][0]
except IndexError:
    print("Error: Could not find expected 'Fixed Prices' columns in Consumption.csv.")
    print("Please check column names like 'Package Holidays - Fixed Prices'.")
    exit()

age_df = age_df.rename(columns={
    package_col_age: 'Packages',
    restaurant_col_age: 'Restaurants',
    accommodation_col_age: 'Accommodation',
    'AGE': 'Age Group' # Assuming 'AGE' is the column for age groups
})
age_df['Total'] = age_df['Packages'] + age_df['Restaurants'] + age_df['Accommodation'] # Calculate total spending

# Process region data
region_df = regions.copy()
# Find the exact column names containing 'Package Holidays - Fixed Prices', etc.
try:
    package_col_region = [col for col in region_df.columns if 'Package Holidays - Fixed Prices' in col][0]
    restaurant_col_region = [col for col in region_df.columns if 'Restaurant Services - Fixed Prices' in col][0]
    accommodation_col_region = [col for col in region_df.columns if 'Accommodation Services - Fixed Prices' in col][0]
except IndexError:
    print("Error: Could not find expected 'Fixed Prices' columns in Region.csv.")
    print("Please check column names like 'Package Holidays - Fixed Prices'.")
    exit()

region_df = region_df.rename(columns={
    package_col_region: 'Packages',
    restaurant_col_region: 'Restaurants',
    accommodation_col_region: 'Accommodation',
    'REGION': 'Region' # Assuming 'REGION' is the column for regions
})
region_df['Total'] = region_df['Packages'] + region_df['Restaurants'] + region_df['Accommodation'] # Calculate total spending

# Define vibrant colors for each spending category
category_colors = {
    'Packages': '#9b59b6',       # Amethyst (Vibrant Purple)
    'Restaurants': '#2ecc71',      # Emerald (Vibrant Green)
    'Accommodation': '#f1c40f'    # Sunflower (Vibrant Yellow)
}

# Define ENGLISH labels for the legend
english_labels = {
    'Packages': 'Package Holidays',
    'Restaurants': 'Restaurants',
    'Accommodation': 'Accommodation'
}

# Fixed maximum value for scaling
fixed_max = 25000

# Function to prepare data for Chart.js format
def prepare_chart_data(df, group_col, display_col=None):
    if display_col is None:
        display_col = group_col

    # Get the list of groups
    groups = df[display_col].tolist()

    # Prepare datasets for each category
    datasets = []

    # Create dataset for Restaurants (will be at bottom layer with lower opacity)
    restaurants_values = df['Restaurants'].tolist()
    restaurants_dataset = {
        'label': english_labels['Restaurants'],
        'data': restaurants_values,
        'backgroundColor': 'rgba(46, 204, 113, 0.5)',   # Green with medium opacity
        'borderColor': '#2ecc71',
        'borderWidth': 2,
        'pointBackgroundColor': '#2ecc71',
        'pointBorderColor': '#fff',
        'pointHoverBackgroundColor': '#fff',
        'pointHoverBorderColor': '#2ecc71',
        'order': 3   # Higher number = drawn first (bottom layer)
    }
    datasets.append(restaurants_dataset)

    # Create dataset for Packages (will be middle layer)
    packages_values = df['Packages'].tolist()
    packages_dataset = {
        'label': english_labels['Packages'],
        'data': packages_values,
        'backgroundColor': 'rgba(155, 89, 182, 0.6)',   # Purple with higher opacity
        'borderColor': '#9b59b6',
        'borderWidth': 2,
        'pointBackgroundColor': '#9b59b6',
        'pointBorderColor': '#fff',
        'pointHoverBackgroundColor': '#fff',
        'pointHoverBorderColor': '#9b59b6',
        'order': 2   # Middle layer
    }
    datasets.append(packages_dataset)

    # Create dataset for Accommodation (will be top layer with highest opacity)
    accommodation_values = df['Accommodation'].tolist()
    accommodation_dataset = {
        'label': english_labels['Accommodation'],
        'data': accommodation_values,
        'backgroundColor': 'rgba(241, 196, 15, 0.61)',   # Yellow with highest opacity
        'borderColor': '#f1c40f',
        'borderWidth': 2,
        'pointBackgroundColor': '#f1c40f',
        'pointBorderColor': '#fff',
        'pointHoverBackgroundColor': '#fff',
        'pointHoverBorderColor': '#f1c40f',
        'order': 1   # Lower number = drawn last (top layer)
    }
    datasets.append(accommodation_dataset)

    return {
        'labels': groups,
        'datasets': datasets
    }

# Prepare data for each chart
socioeconomic_data = prepare_chart_data(socio_df, 'Group', 'Group_EN')
age_data = prepare_chart_data(age_df, 'Age Group')
region_data = prepare_chart_data(region_df, 'Region')

# Create a dictionary with all chart data
chart_data = {
    'socioeconomic': socioeconomic_data,
    'age': age_data,
    'region': region_data
}

# Convert to JSON for use in the HTML
chart_data_json = json.dumps(chart_data)

# Create HTML template for the interactive radar chart
html_template = '''<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no">
    <title>Danish Travel Spending Patterns</title>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/Chart.js/3.9.1/chart.min.js"></script>
    <style>
        /* Ensure html and body fill the iframe and hide overflow */
        html, body {
            height: 100%; /* Makes html and body take 100% of iframe's height */
            width: 100%;
            margin: 0;
            padding: 0;
            overflow: hidden; /* Hides any scrollbars within the iframe */
        }
        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            padding: 0;
            background-color: transparent;

            /* Flexbox for vertical layout */
            display: flex;
            flex-direction: column;
        }
        .chart-container {
            position: relative;
            width: 100%;
            margin: 0 auto; /* Centers horizontally if width is less than 100%, but width is 100% here */
            background-color: white;
            padding: 0;
            overflow: hidden; /* Keep for internal canvas overflow */

            /* Flexbox for remaining space */
            flex-grow: 1; /* Allows it to take up all available vertical space */
            min-height: 0; /* Crucial for flex items to shrink and prevent overflow */
        }
        /* Ensure fixed height elements don't cause overflow by not shrinking */
        h1, .subtitle, .tab-container {
            flex-shrink: 0; /* Prevents these elements from shrinking */
        }

        /* Original styles for aesthetic elements */
        h1 {
            text-align: center;
            color: #333;
            margin: 10px 0;
        }
        .subtitle {
            text-align: center;
            color: #666;
            margin-bottom: 10px;
            font-style: italic;
        }
        .tab-container {
            display: flex;
            justify-content: center;
            margin-bottom: 20px;
        }
        .tab {
            padding: 12px 24px;
            background-color: #e0e0e0;
            border: none;
            cursor: pointer;
            font-size: 16px;
            font-weight: 500;
            transition: all 0.3s ease;
            border-radius: 4px;
            margin: 0 8px;
        }
        .tab.active {
            background-color: #3498db;
            color: white;
            box-shadow: 0 2px 8px rgba(52, 152, 219, 0.4);
        }
        .tab:hover:not(.active) {
            background-color: #d0d0d0;
        }
        /* Canvas sizing within chart-container */
        canvas {
            display: block; /* Ensures no extra space below the canvas */
            width: 100%; /* Chart.js responsive will adapt to parent width */
            height: 100%; /* Chart.js responsive will adapt to parent height */
        }
        @media (max-width: 768px) {
            .tab {
                padding: 8px 16px;
                font-size: 14px;
            }
        }
    </style>
</head>
<body>
    <h1>Danish Travel Spending Patterns</h1>
    <p class="subtitle">Explore how different Danish demographics allocate their travel budgets</p>

    <div class="tab-container">
        <button class="tab active" data-category="socioeconomic">Socioeconomic Groups</button>
        <button class="tab" data-category="age">Age Groups</button>
        <button class="tab" data-category="region">Regions</button>
    </div>

    <div class="chart-container">
        <canvas id="radarChart"></canvas>
    </div>

    <script>
        // Store chart instance for updating
        let radarChart;

        // Chart data from Python
        const chartData = CHART_DATA_PLACEHOLDER;

        // Fixed maximum value for scaling
        const fixedMaxValue = 25000;

        // Chart configuration
        const config = {
            type: 'radar',
            data: chartData.socioeconomic, // Initial dataset
            options: {
                responsive: true,
                maintainAspectRatio: false,
                plugins: {
                    legend: {
                        position: 'top',
                        reverse: true, // Reverse legend order to match visual order
                        labels: {
                            font: {
                                size: 14
                            },
                            padding: 20
                        }
                    },
                    title: {
                        display: true,
                        text: 'Socioeconomic Groups: Travel Spending Distribution',
                        font: {
                            size: 18,
                            weight: 'bold'
                        },
                        padding: {
                            top: 10,
                            bottom: 30
                        }
                    },
                    tooltip: {
                        callbacks: {
                            label: function(context) {
                                return ` ${context.dataset.label}: ${context.raw.toFixed(2)}`; // Display raw consumption value
                            }
                        }
                    }
                },
                scales: {
                    r: {
                        angleLines: {
                            color: '#cccccc'
                        },
                        grid: {
                            color: '#cccccc'
                        },
                        pointLabels: {
                            font: {
                                size: 14
                            },
                            color: '#555555'
                        },
                        ticks: {
                            backdropColor: 'transparent',
                            color: '#777777',
                            showLabelBackdrop: false,
                            font: {
                                size: 10
                            },
                            callback: function(value) {
                                return value.toFixed(0); // Format ticks as whole numbers
                            },
                            max: fixedMaxValue, // Set the fixed maximum value for the scale
                            min: 0,
                            stepSize: 5000 // Optional: Add step size for better readability
                        },
                        suggestedMin: 0,
                        suggestedMax: fixedMaxValue // Ensure suggested max is also set
                    }
                },
                elements: {
                    line: {
                        tension: 0.1 // Smoother lines
                    }
                }
            }
        };

        // Initialize the chart
        window.onload = function() {
            const ctx = document.getElementById('radarChart').getContext('2d');
            radarChart = new Chart(ctx, config);

            // Add event listeners to tabs using data attributes
            document.querySelectorAll('.tab').forEach(tab => {
                tab.addEventListener('click', function() {
                    const category = this.getAttribute('data-category');
                    showChart(category);
                });
            });
        };

        // Function to switch between datasets
        function showChart(category) {
            const titles = {
                socioeconomic: 'Socioeconomic Groups: Travel Spending Distribution',
                age: 'Age Groups: Travel Spending Distribution',
                region: 'Regions: Travel Spending Distribution'
            };

            // Update chart data
            radarChart.data = chartData[category];

            // Update chart title
            radarChart.options.plugins.title.text = titles[category];

            // Update the scale's max and suggestedMax when switching charts
            radarChart.options.scales.r.max = fixedMaxValue;
            radarChart.options.scales.r.suggestedMax = fixedMaxValue;

            // Update chart
            radarChart.update();

            // Update active tab
            document.querySelectorAll('.tab').forEach(tab => {
                if (tab.getAttribute('data-category') === category) {
                    tab.classList.add('active');
                } else {
                    tab.classList.remove('active');
                }
            });
        }
    </script>
</body>
</html>
'''

# Replace the placeholder with the actual JSON data
html_content = html_template.replace('CHART_DATA_PLACEHOLDER', chart_data_json)

# Write the HTML to a file
output_file = '../assets/danish_travel_radar_interactive.html'
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(html_content)

print(f"Interactive radar visualization created successfully and optimized for web fit with consistent scaling up to {fixed_max} at '{output_file}'!")

Interactive radar visualization created successfully and optimized for web fit with consistent scaling up to 25000 at '../assets/danish_travel_radar_interactive.html'!
